In [43]:
import pandas as pd
import numpy as np

In [44]:
from sklearn.metrics import mean_squared_error

In [45]:
import os

In [46]:
y_true = pd.read_csv('datasets/test_set.csv')['cpi_pct'].to_numpy()

In [47]:
prediction = []
for file_name in os.listdir('predictions/'):
    
    if file_name[:3] == 'pca':
        continue
    
    y_pred = pd.read_csv(f'predictions/{file_name}', index_col=0).to_numpy()
    
    name = file_name.rsplit('.', 1)[0]
    rmse =  mean_squared_error(y_true, y_pred, squared=False)
    print(name, "rmse:", round(rmse, 7))
    prediction.append({'model': name, 'rmse': rmse})

benchmark1_mean rmse: 0.0032184
benchmark2_prev rmse: 0.0026672
benchmark3_rf rmse: 0.0020702
bma rmse: 0.0020434
lasso_bic rmse: 0.0021458
lasso_cv rmse: 0.0021016
lasso_cvshuffled rmse: 0.0030599
lasso_ebic rmse: 0.0021458
ols rmse: 0.0044418
ridge_cv rmse: 0.0021375


In [50]:
sorting = ['benchmark1_mean', 'benchmark2_prev', 'benchmark3_rf', 
           'ols','ridge_cv', 'lasso_cv', 'lasso_cvshuffled',
           'lasso_bic', 'lasso_ebic', 'bma']

In [51]:
score_df = pd.DataFrame(prediction).set_index('model')

In [60]:
score_df['diff to best'] =  score_df.rmse - score_df.rmse.min()

In [64]:
score_df.loc[sorting, :].to_csv('results/1970_2023_model_scores.csv')
score_df.sort_values('rmse').to_csv('results/1970_2023_model_scores_sorted.csv')

2000-2023 model:

- benchmark1_mean rmse: 0.0026948
- benchmark2_prev rmse: 0.0034082
- bma rmse: 0.002297
- lasso_bic rmse: 0.0023243
- lasso_ebic rmse: 0.0023711
- lasso_cv rmse: 0.0027338
- lasso_cvshuffled rmse: 0.0027338
- ols rmse: 0.0279417
- ridge_cv rmse: 0.0024787

1970-2023 model:

- benchmark1_mean rmse: 0.0032184
- benchmark2_prev rmse: 0.0026672
- benchmark3_rf rmse: 0.0020702
- bma rmse: 0.0020429
- lasso_bic rmse: 0.0021015
- lasso_cv rmse: 0.0021016
- lasso_cvshuffled rmse: 0.0019987
- lasso_ebic rmse: 0.0021058
- ols rmse: 0.0030478
- ridge_cv rmse: 0.0021375

1970-2023 model (PCA):

- benchmark1_mean rmse: 0.0026948
- benchmark2_prev rmse: 0.0034082
- benchmark3_rf rmse: 0.0020702
- bma rmse: 0.002297
- lasso_bic rmse: 0.0023243
- lasso_ebic rmse: 0.0023711
- lasso_cv rmse: 0.0027338
- lasso_cvshuffled rmse: 0.0027338
- ols rmse: 0.0279417
- ridge_cv rmse: 0.0024787